In [6]:
import os
import pandas as pd
from pypdf import PdfReader
import re

In [2]:
# get the location of the pdf files
csv_path = 'metadata.csv'
df = pd.read_csv(csv_path)

In [3]:
# create a directory to store the plaintext files
output_dir = './plaintext'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [4]:
# print the current working directory
print('Current working directory:', os.getcwd())

Current working directory: /home/sam/Simply-Business-Code


The complaint
What happened
What I've decided and why
My final decision

In [54]:
def clean_text(text):
    # Delete the PDF identification
    text = re.sub(r'^DRN-\d+\n', '', text)
    # Change "’" to "'"
    text = re.sub(r'’', "'", text)
    # Delete the non-ASCII characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    # Delete the extra spaces
    text = re.sub(r'\s+', ' ', text)
    
    # Add new lines before specific keywords
    text = text.replace("The complaint ", 'The complaint\n')
    text = text.replace("What happened ", '\nWhat happened\n')
    text = text.replace("What I've decided and why ", "\nWhat I've decided and why\n")
    
    # Remove "My final decision" and everything after it
    text = re.sub(r'My final decision.*', '', text, flags=re.DOTALL)
    
    return text

In [56]:
# iterate through each row to get the PDF file path from 'location' column
for index, row in df.iterrows():
    pdf_path = row['location']

    # extract text from the PDF file
    try:
        reader = PdfReader(pdf_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        
        # clean the text
        text = clean_text(text)

        # create a plaintext file with the same name as the PDF file
        txt_filename = os.path.basename(pdf_path).replace('.pdf', '.txt')
        txt_path = os.path.join(output_dir, txt_filename)
        
        # write the text to the file
        with open(txt_path, 'w', encoding='utf-8') as txt_file:
            txt_file.write(text)
    
    except Exception as e:
        print(f"Error processing {pdf_path}: {e}")